In [ ]:
import os


## Download PlanCLEF data

In [ ]:
plant_clef_test_url = "http://otmedia.lirmm.fr/LifeCLEF/PlantCLEF2015/Packages/TestPackage/PlantCLEF2015TestDataWithAnnotations.tar.gz"
plant_clef_test_path = "data/PlantCLEF2015TestDataWithAnnotations.tar.gz"
plant_clef_test_folder = "data/PlantCLEF2015TestDataWithAnnotations"
if not os.path.exists(plant_clef_test_path):    
    !wget -O $plant_clef_test_path $plant_clef_test_url
if not os.path.exists(plant_clef_test_folder):
    !tar -zxf $plant_clef_test_path

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
# display plots in this notebook
%matplotlib inline

In [ ]:
import sklearn.datasets

In [ ]:
mnist = sklearn.datasets.fetch_mldata('MNIST original')
data = np.array(mnist['data'].reshape(70000, 1, 28,28), dtype='float32')
labels = np.array(mnist['target'], dtype='float32')
plt.figure(figsize=(10,10))
plt.imshow(np.squeeze(np.concatenate(data[0:60000:7000,:,:], axis=2)), cmap='gray');
plt.axis('off');

# Inject data from memory to Caffe

In [ ]:
import caffe

In [ ]:
from caffe import layers as L, params as P

def lenet(batch_size):
    # our version of LeNet: a series of linear and simple nonlinear transformations
    n = caffe.NetSpec()
    
    n.data, n.label = L.MemoryData(batch_size=batch_size, channels=1, height=28, width=28, transform_param=dict(scale=1./255), ntop=2)
#    n.label = L.MemoryData(batch_size=batch_size, channels=1, height=1, width=1)
    
#    L.Data(batch_size=batch_size, backend=P.Data.MemoryData)
#    n.data, n.label = L.Data(batch_size=batch_size, backend=P.Data.LMDB, source=lmdb,
#                             transform_param=dict(scale=1./255), ntop=2)
    
    n.conv1 = L.Convolution(n.data, kernel_size=5, num_output=20, weight_filler=dict(type='xavier'))
    n.pool1 = L.Pooling(n.conv1, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.conv2 = L.Convolution(n.pool1, kernel_size=5, num_output=50, weight_filler=dict(type='xavier'))
    n.pool2 = L.Pooling(n.conv2, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.fc1 =   L.InnerProduct(n.pool2, num_output=500, weight_filler=dict(type='xavier'))
    n.relu1 = L.ReLU(n.fc1, in_place=True)
    n.score = L.InnerProduct(n.relu1, num_output=10, weight_filler=dict(type='xavier'))
    n.loss =  L.SoftmaxWithLoss(n.score, n.label)
    
    return n.to_proto()
    



test_net = lenet(100)
    
with open('lenet_auto_train.prototxt', 'w') as f:
    f.write(str(lenet(60)))
    
with open('lenet_auto_test.prototxt', 'w') as f:
    f.write(str(lenet(100)))


In [1]:
def lenet(lmdb, batch_size):
    # our version of LeNet: a series of linear and simple nonlinear transformations
    n = caffe.NetSpec()
    
    n.data, n.label = L.Data(batch_size=batch_size, backend=P.Data.LMDB, source=lmdb,
                             transform_param=dict(scale=1./255), ntop=2)
    
    n.conv1 = L.Convolution(n.data, kernel_size=5, num_output=20, weight_filler=dict(type='xavier'))
    n.pool1 = L.Pooling(n.conv1, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.conv2 = L.Convolution(n.pool1, kernel_size=5, num_output=50, weight_filler=dict(type='xavier'))
    n.pool2 = L.Pooling(n.conv2, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.fc1 =   L.InnerProduct(n.pool2, num_output=500, weight_filler=dict(type='xavier'))
    n.relu1 = L.ReLU(n.fc1, in_place=True)
    n.score = L.InnerProduct(n.relu1, num_output=10, weight_filler=dict(type='xavier'))
    n.loss =  L.SoftmaxWithLoss(n.score, n.label)
    
    return n.to_proto()
    
with open('lenet_auto_train.prototxt', 'w') as f:
    f.write(str(lenet('data/plantclef_train_lmdb', 64)))
    
with open('lenet_auto_test.prototxt', 'w') as f:
    f.write(str(lenet('data/plantclef_test_lmdb', 100)))

IOError: [Errno 2] No such file or directory: 'mnist/lenet_auto_train.prototxt'

In [ ]:
#test_net
#train_net = caffe.Net('lenet_auto_train.prototxt', caffe.TRAIN)

In [ ]:
solver_path = 'solver.prototxt'
solver_proto = """\
# The train/test net protocol buffer definition
train_net: "lenet_auto_train.prototxt"
test_net: "lenet_auto_test.prototxt"
# test_iter specifies how many forward passes the test should carry out.
# In the case of MNIST, we have test batch size 100 and 100 test iterations,
# covering the full 10,000 testing images.
test_iter: 100
# Carry out testing every 500 training iterations.
test_interval: 500
# The base learning rate, momentum and the weight decay of the network.
base_lr: 0.01
momentum: 0.9
weight_decay: 0.0005
# The learning rate policy
lr_policy: "inv"
gamma: 0.0001
power: 0.75
# Display every 100 iterations
display: 100
# The maximum number of iterations
max_iter: 10000
# snapshot intermediate results
snapshot: 5000
snapshot_prefix: "lenet"
"""

open(solver_path,'w').write(solver_proto)


In [ ]:
caffe.set_device(0)
caffe.set_mode_gpu()

In [ ]:
solver = None
solver = caffe.SGDSolver(solver_path)

Assingning memory buffers to the input layers

In [ ]:
solver.net.set_input_arrays(data[0:60000], 
                            labels[0:60000])

In [ ]:
solver.step(1)

In [ ]:
data.shape